# Predicting survival in the Titanic

We train an ML model on the manifest from the Titanic to learn to predict who would survive.

We start by loading the manifest dataset into a Pandas dataframe.

In [39]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Load Titanic dataset
data = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Dataset

We start by removing the features that do not seem to influence the probability of survival.

Our goal is to predict the value in the column `Survived`. We assign this to the target variable `y` and drop it from the features, as otherwise the prediction task would be trivial.

Create the objects `X` and `y` from the Titanic dataset.

In [40]:
y = data["Survived"]
X = data.drop("Survived", axis='columns') 

## Imputing missing values

You can see in the table above that some ages and fares are missing. This prevents some ML algorithms from working. To address this, we replace missing values with the median age and the median fare.

In [41]:
numerical_imputer = SimpleImputer(strategy='median')
numerical_imputer.fit(X[['Age', 'Fare']])
X[['Age', 'Fare']] = numerical_imputer.transform(X[['Age', 'Fare']])

C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprec

For values that are non-numeric, like those in columns `Sex`, `Embarked`, and `Cabin`, we impute missing values with the most common value. This can be done with an imputer whose strategy is `most_frequent`. Try by yourself to create this imputer.

In [42]:
categorical_imputer = SimpleImputer(strategy="most_frequent")
categorical_imputer.fit(X[['Sex', 'Embarked', 'Cabin']])
X[['Sex', 'Embarked', 'Cabin']] = categorical_imputer.transform(X[['Sex', 'Embarked', 'Cabin']])
X

C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprec

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,B96 B98,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,B96 B98,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,B96 B98,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,B96 B98,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Encoding categorical values

Values that are non-numeric need to be transformed into numeric, as ML algorithms work only numeric values. We can do this using the function `get_dummies` from Pandas. This does one-hot encoding.

In [43]:
categorical_features = ["Sex", "Embarked", "Cabin"]
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)
X

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Sex_male,Embarked_Q,...,Cabin_E8,Cabin_F E69,Cabin_F G63,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T
0,1,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,True,False,...,False,False,False,False,False,False,False,False,False,False
887,888,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,False,False,...,False,False,False,False,False,False,False,False,False,False
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",28.0,1,2,W./C. 6607,23.4500,False,False,...,False,False,False,False,False,False,False,False,False,False
889,890,1,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,True,False,...,False,False,False,False,False,False,False,False,False,False


## Training the model

We are now ready to train the model. Train a classification tree and validate its accuracy.

In [47]:
#X = X.drop("Name", axis="columns")
#X = X.drop("Ticket", axis="columns")
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

grid = {'max_depth': range(3,10)}
tree = DecisionTreeClassifier(max_depth=5)

model = GridSearchCV(tree, param_grid=grid, cv=5)
model.fit(X_train, y_train)

acc_train = accuracy_score(y_train, model.predict(X_train))
acc_test = accuracy_score(y_test, model.predict(X_test))
print("Accuracy of classif. tree on train data: {:.3f}".format(acc_train))
print("Accuracy of classif. tree on test data: {:.3f}".format(acc_test))

C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Che

C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprec

Accuracy of classif. tree on train data: 0.833
Accuracy of classif. tree on test data: 0.838


C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\diabomba\anaconda3\envs\ETH_CAS_AIS_2024\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprec

Observe how this decision tree can now predict survival with an accuracy of 78%.

In [48]:
print("Accuracy of classif. tree on train data: {:.3f}".format(acc_train))
print("Accuracy of classif. tree on test data: {:.3f}".format(acc_test))

Accuracy of classif. tree on train data: 0.833
Accuracy of classif. tree on test data: 0.838
